# Demo of SED and photometry calculations with DSPS

author: Luca Tortorelli

last run successfully: May 10, 2024

This notebook demonstrates some basic usage of the rail_dsps library. In particular, for a galaxy with some arbitrary star formation history, we'll see how to calculate its restframe SED, and its absolute and apparent magnitude in some band.

SPS calculations are based on a set of template SEDs of simple stellar populations (SSPs). Supplying such templates is outside the planned scope of the DSPS package, and so they will need to be retrieved from some other library. For example, the FSPS library supplies such templates in a convenient form.

In case the user is not aware of how to generate these templates, the code automatically downloads a pre-defined library from NERSC.

### SingleSedModeler

The SingleSedModeler class allows the user to generate a single rest-frame SED at the time with rail_dsps.

Some example galaxy properties that are required to create a single SED model are generated via this notebook and stored into an hdf5 table. The required galaxy properties are:

- galaxy redshifts
- the grid of cosmic time over which the star-formation history of galaxies is evaluated in units of Gyr
- the star-formation history of galaxies in units of Msun/yr
- galaxy metallicity at the time of observation in units of log10(Z)
- log normal scatter of the galaxy metallicity at the time of observation


In [1]:
import os
import rail.dsps
from rail.core.stage import RailStage
from rail.core.data import Hdf5Handle
import numpy as np
import h5py

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

RAIL_DSPS_DIR = os.path.abspath(os.path.join(os.path.dirname(rail.dsps.__file__), '..'))
default_rail_fsps_files_folder = os.path.join(RAIL_DSPS_DIR, 'examples_data/creation_data/data/dsps_default_data')
trainFile = os.path.join(default_rail_fsps_files_folder, 'input_galaxy_properties_dsps.hdf5')

In [ ]:
n_galaxies = 100

redshift = np.linspace(0.1,1.0,num=n_galaxies)

gal_t_table = np.linspace(0.05, 13.8, 100) # age of the universe in Gyr
gal_sfr_table = np.random.uniform(0, 10, gal_t_table.size) # SFR in Msun/yr

gal_lgmet = -2.0 # log10(Z)
gal_lgmet_scatter = 0.2 # lognormal scatter in the metallicity distribution function

cosmic_time_grid = np.full((n_galaxies, len(gal_t_table)), gal_t_table)
star_formation_history = np.full((n_galaxies, len(gal_sfr_table)), gal_sfr_table)
stellar_metallicity = np.full(n_galaxies, gal_lgmet)
stellar_metallicity_scatter = np.full(n_galaxies, gal_lgmet_scatter)

with h5py.File(trainFile, 'w') as h5table:
    h5table.create_dataset(name='redshifts', data=redshift)
    h5table.create_dataset(name='cosmic_time_grid', data=cosmic_time_grid)
    h5table.create_dataset(name='star_formation_history', data=star_formation_history)
    h5table.create_dataset(name='stellar_metallicity', data=stellar_metallicity)
    h5table.create_dataset(name='stellar_metallicity_scatter', data=stellar_metallicity_scatter)

In [ ]:
training_data = DS.read_file("training_data", Hdf5Handle, trainFile)

The user is also required to provide the template SSPs with which rail_dsps generates its rest-frame SEDs. Leaving it blank or to a non-existing file will lead rail_dsps to generate the default templates from NERSC.

In [ ]:
dspssinglesedmodeler = rail.dsps.DSPSSingleSedModeler.make_stage(name='DSPSSingleSedModeler',
                                                                 ssp_templates_file=os.path.join(RAIL_DSPS_DIR,'examples_data/creation_data/data/dsps_default_data/ssp_data_fsps_v3.2_lgmet_age.h5'),
                                                                 redshift_key='redshifts',
                                                                 cosmic_time_grid_key='cosmic_time_grid',
                                                                 star_formation_history_key='star_formation_history',
                                                                 stellar_metallicity_key='stellar_metallicity',
                                                                 stellar_metallicity_scatter_key='stellar_metallicity_scatter',
                                                                 restframe_sed_key='restframe_seds', default_cosmology=True,
                                                                 min_wavelength=250, max_wavelength=12000)

In [ ]:
dspssinglesedmodel = dspssinglesedmodeler.fit_model(input_data=training_data)

In [ ]:
dspssinglesedmodel.data

### PopulationSedModeler

The PopulationSedModeler class works in a similar way as the SingleSedModeler class, but allows the user to generate a population of rest-frame SEDs using the native parallelization capabilities of jax.


In [ ]:
import os
import rail.dsps
from rail.core.stage import RailStage
from rail.core.data import Hdf5Handle
import numpy as np
import h5py

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

default_rail_fsps_files_folder = os.path.join(RAIL_DSPS_DIR, 'examples_data/creation_data/data/dsps_default_data')
trainFile = os.path.join(default_rail_fsps_files_folder, 'input_galaxy_properties_dsps.hdf5')

In [ ]:
n_galaxies = 100

redshift = np.linspace(0.1,1.0,num=n_galaxies)

gal_t_table = np.linspace(0.05, 13.8, 100) # age of the universe in Gyr
gal_sfr_table = np.random.uniform(0, 10, gal_t_table.size) # SFR in Msun/yr

gal_lgmet = -2.0 # log10(Z)
gal_lgmet_scatter = 0.2 # lognormal scatter in the metallicity distribution function

cosmic_time_grid = np.full((n_galaxies, len(gal_t_table)), gal_t_table)
star_formation_history = np.full((n_galaxies, len(gal_sfr_table)), gal_sfr_table)
stellar_metallicity = np.full(n_galaxies, gal_lgmet)
stellar_metallicity_scatter = np.full(n_galaxies, gal_lgmet_scatter)

with h5py.File(trainFile, 'w') as h5table:
    h5table.create_dataset(name='redshifts', data=redshift)
    h5table.create_dataset(name='cosmic_time_grid', data=cosmic_time_grid)
    h5table.create_dataset(name='star_formation_history', data=star_formation_history)
    h5table.create_dataset(name='stellar_metallicity', data=stellar_metallicity)
    h5table.create_dataset(name='stellar_metallicity_scatter', data=stellar_metallicity_scatter)

In [ ]:
training_data = DS.read_file("training_data", Hdf5Handle, trainFile)

In [ ]:
dspspopulationsedmodeler = rail.dsps.DSPSPopulationSedModeler.make_stage(name='DSPSPopulationSedModeler',
                                                                         ssp_templates_file=os.path.join(RAIL_DSPS_DIR,
                                                                         'examples_data/creation_data/data/dsps_default_data/ssp_data_fsps_v3.2_lgmet_age.h5'),
                                                                         redshift_key='redshifts',
                                                                         cosmic_time_grid_key='cosmic_time_grid',
                                                                         star_formation_history_key='star_formation_history',
                                                                         stellar_metallicity_key='stellar_metallicity',
                                                                         stellar_metallicity_scatter_key='stellar_metallicity_scatter',
                                                                         restframe_sed_key='restframe_seds', default_cosmology=True,
                                                                         min_wavelength=250, max_wavelength=12000)

In [ ]:
dspspopulationsedmodel = dspspopulationsedmodeler.fit_model(input_data=training_data)

In [ ]:
dspspopulationsedmodel.data

### DSPSPhotometryCreator

This class allows the user to generate model photometry by computing the absolute and apparent magnitudes of galaxies from their input rest-frame SEDs. Although DSPSPopulationSedModeler generates the rest-frame SEDs that are needed for this class, the user can supply whatever external SED provided that the units are in Lsun/Hz.

Generating the observed photometry with DSPS is simple and requires only few input from the user. The required input are:
- the redshift dataset keyword of the hdf5 table containing the rest-frame SEDs output from the DSPSPopulationSedModeler
- the rest-frame SEDs dataset keyword of the hdf5 table containing the rest-frame SEDs output from the DSPSPopulationSedModeler
- the absolute and apparent magnitudes dataset keyword of the output hdf5 table
- the folder path containing the filter bands
- the name of the filter bands in order of increasing wavelength
- the minimum and maximum wavelength used in the DSPSPopulationSedModeler class to generate the rest-frame SEDs
- the path to the SSP template files
- a boolean keyword to use (True) the default cosmology in DSPS.

If the latter keyword is set to False, then the user has to manually provide the values of Om0, w0, wa and h in the .sample function.

In [ ]:
import os
import rail.dsps
from rail.core.stage import RailStage
from rail.core.data import Hdf5Handle

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

trainFile = 'model_DSPSPopulationSedModeler.hdf5'

In [ ]:
training_data = DS.read_file("training_data", Hdf5Handle, trainFile)

In [ ]:
dspsphotometrycreator = rail.dsps.DSPSPhotometryCreator.make_stage(name='DSPSPhotometryCreator',
                                                         redshift_key='redshifts',
                                                         restframe_sed_key='restframe_seds',
                                                         absolute_mags_key='rest_frame_absolute_mags',
                                                         apparent_mags_key='apparent_mags',
                                                         filter_folder=os.path.join(RAIL_DSPS_DIR,
                                                         'examples_data/creation_data/data/dsps_default_data/filters'),
                                                         instrument_name='lsst',
                                                         wavebands='u,g,r,i,z,y',
                                                         min_wavelength=250, max_wavelength=12000,
                                                         ssp_templates_file=os.path.join(RAIL_DSPS_DIR,
                                                         'examples_data/creation_data/data/dsps_default_data/ssp_data_fsps_v3.2_lgmet_age.h5'),
                                                         default_cosmology=True)

In [ ]:
dspsphotometry = dspsphotometrycreator.sample(input_data=training_data)

In [ ]:
dspsphotometry.data